In [1]:
!pip install PyMuPDF nltk  Pillow python-docx openai  scikit-image scipy numpy 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 54.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 10.2 MB/s eta 0:00:00


In [2]:
!pip install  gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 23.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 73.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
import gradio
print(gradio.__version__)


5.20.0


In [ ]:

import os
import fitz
import re
import nltk
import time
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO
from PIL import Image
import requests
import base64
import json
from openai import OpenAI
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH


nltk.download('punkt', quiet=True)


openrouter_api_key = "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"
qwen_api_key = "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" 


client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
)

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file"""
    doc = fitz.open(pdf_path)
    text = ""
    
    for page_num, page in enumerate(doc):
        # Extract text
        page_text = page.get_text("text")
        page_text = re.sub(r'\s+', ' ', page_text)  # Normalize spaces
        text += page_text.strip() + "\n\n"
                
    return text.strip()

def load_figures_from_folder(figures_folder):
    """Load figures from a folder"""
    figures = []
    
    
    if not os.path.exists(figures_folder):
        print(f"Figures folder not found: {figures_folder}")
        return figures
    
  
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']
    image_files = [f for f in os.listdir(figures_folder) 
                  if os.path.isfile(os.path.join(figures_folder, f)) and 
                  any(f.lower().endswith(ext) for ext in image_extensions)]
    
    
    for img_file in image_files:
        try:
            img_path = os.path.join(figures_folder, img_file)
            image = Image.open(img_path)
            
            
            filename = os.path.splitext(img_file)[0]
            caption = ""
            
            
            fig_match = re.search(r'(figure|fig)[_\-\s]*(\d+)[_\-\s]*(.*)', filename, re.IGNORECASE)
            if fig_match:
                fig_num = fig_match.group(2)
                fig_caption = fig_match.group(3).replace('_', ' ').replace('-', ' ')
                caption = f"Figure {fig_num}: {fig_caption}"
            
            
            figures.append({
                'image': image,
                'filename': img_file,
                'is_figure': True, 
                'caption': caption,
                'page': 0  
            })
            
        except Exception as e:
            print(f"Error loading image {img_file}: {e}")
    
    print(f"Loaded {len(figures)} figures from folder")
    return figures

def analyze_image_with_qwen(image, context=""):
    """Use Qwen VL through OpenRouter to analyze an image"""
    
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
    
    
    if context:
        prompt = f"Analyze this figure from a textbook and explain its significance. Context: {context}"
    else:
        prompt = "Describe this figure from a textbook and explain what it represents in educational context."
    
    
    openai_client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=qwen_api_key,
    )
    
    try:
        
        completion = openai_client.chat.completions.create(
            model="qwen/qwen-vl-plus:free",  
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{img_base64}"
                            }
                        }
                    ]
                }
            ]
        )
        
        return completion.choices[0].message.content
        
    except Exception as e:
        print(f"Error in Qwen VL API call: {e}")
        # Fallback description if the API call fails
        return "A diagram from the textbook."

def clean_text(text, for_qa=False):
    """
    Clean text with different patterns based on usage (summary or Q&A)
    """
    if for_qa:
        
        text = re.sub(r'Activity\s+\d+\.\d+.*?(\n|$)', '', text, flags=re.DOTALL)
        text = re.sub(r'Figure\s+\d+\.\d+.*?\n', '', text, flags=re.IGNORECASE)
        text = re.sub(r'\b\d{4}-\d{2}\b', '', text)
        text = re.sub(r'\n+', '\n', text).strip()
    else:
        
        text = re.sub(r'Activity\s+\d+\.\d+.*?(\n|$)', '', text, flags=re.DOTALL)
        text = re.sub(r'Q\s?U\s?E\s?S\s?T\s?I\s?O\s?N\s?S[\s\S]*?(\n|$)', '', text, flags=re.DOTALL)
        text = re.sub(r'Figure\s+\d+\.\d+.*?\n', '', text, flags=re.IGNORECASE)
        text = re.sub(r'\b\d{4}-\d{2}\b', '', text)
        text = re.sub(r'\n+', '\n', text).strip()

    
    text = re.sub(r'\\\((.*?)\\\)', lambda m: m.group(1).replace('\\', '').replace('_', ''), text)
    text = re.sub(r'\\\[(.*?)\\\]', lambda m: m.group(1).replace('\\', '').replace('_', ''), text)
    text = re.sub(r'\\text\{(.*?)\}', r'\1', text) 
    text = re.sub(r'\\left\((.*?)\\right\)', r'(\1)', text) 
    text = re.sub(r'\\rightarrow', r'→', text)  
    text = re.sub(r'_\{(.*?)\}', r'_\1', text)  
    text = re.sub(r'\^\{(.*?)\}', r'^\1', text)  
    
    return text

def extract_chapters(text, for_qa=False):
    chapter_pattern = r"(\d+)\s+CHAPTER([\s\S]*?)(?=\d+\s+CHAPTER|\Z)"
    matches = re.findall(chapter_pattern, text, re.DOTALL)
    
    chapter_dict = {}
    for number, content in matches:
        chapter_dict[number] = clean_text(content, for_qa=for_qa)
    
    return chapter_dict

def split_text_into_chunks(text, max_tokens=1000):
    sentences = nltk.sent_tokenize(text)
    chunks, current_chunk, current_length = [], [], 0
    
    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk, current_length = [sentence], sentence_length
        else:
            current_chunk.append(sentence)
            current_length += sentence_length
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def summarize_text(text):
    try:
        prompt = """
        Create a structured academic summary following these guidelines:
        1. Start with key concepts and fundamental principles
        2. Present main topics in a logical sequence
        3. Include important definitions and explanations
        4. Highlight significant examples and applications
        5. Use clear, academic language
        6. Avoid any personal thoughts or meta-commentary
        7. Keep a formal, educational tone throughout
        8. Format main topics in **bold** using Markdown syntax
        9. Write all equations in plain text format (like "2H2 + O2 → 2H2O") not LaTeX format
        
        The summary should be suitable for a textbook chapter and avoid any informal language or thinking process phrases.
        """
        
        completion = client.chat.completions.create(
            model="deepseek/deepseek-r1:free",
            messages=[{
                "role": "user", 
                "content": f"{prompt}\n\nContent to summarize: {text}"
            }],
            temperature=0.3
        )
        
        summary = completion.choices[0].message.content.strip()
        
        
        cleanup_patterns = [
            (r'^(In this chapter|This chapter|Let me|I will).*?[.]\s*', ''),
            (r'(The answer is|This means that|Thus|Therefore|Hence|So|To summarize|In conclusion)[,:](.*?)$', r'\2'),
            (r'I (need to|will|should|can).*?[.!?]\s*', ''),
            (r'Let\'s.*?[.!?]\s*', ''),
            (r'Based on.*?[.!?]\s*', ''),
            (r'Looking at.*?[.!?]\s*', ''),
            (r'First,|Second,|Third,|Finally,', '')
        ]
        
        for pattern, replacement in cleanup_patterns:
            summary = re.sub(pattern, replacement, summary, flags=re.IGNORECASE)
            
        return summary.strip()
    except Exception as e:
        return f"Summarization Error: {e}"

def summarize_chapter(number, content, chapter_images=None):
    if not content.strip():
        return f"**CHAPTER {number}**\n\nNo content available for summary.", []
    
    chunks = split_text_into_chunks(content, max_tokens=2000)
    chunk_summaries = []
    
    for chunk in chunks:
        summary = summarize_text(chunk)
        if summary and not summary.startswith("Summarization Error"):
            chunk_summaries.append(summary)
    
    if len(chunk_summaries) > 1:
        final_summary_prompt = """
        Create a comprehensive 3-page chapter summary that:
        1. Begins with an overview of main concepts
        2. Presents topics in a clear, logical order
        3. Explains key theories and principles
        4. Includes relevant examples and applications
        5. Connects related concepts
        6. Maintains academic language throughout
        7. Avoids repetition and meta-commentary

        Format as clear paragraphs with proper transitions.
        """
        
        try:
            completion = client.chat.completions.create(
                model="deepseek/deepseek-r1:free",
                messages=[{
                    "role": "user",
                    "content": f"{final_summary_prompt}\n\nContent to summarize: {' '.join(chunk_summaries)}"
                }],
                temperature=0.3
            )
            final_summary = completion.choices[0].message.content.strip()
        except Exception:
            final_summary = "\n\n".join(chunk_summaries)
    else:
        final_summary = chunk_summaries[0] if chunk_summaries else "No summary available."
    
    
    processed_images = []
    if chapter_images and len(chapter_images) > 0:
        
        filtered_images = [img for img in chapter_images if img['is_figure']]
        if not filtered_images:
            filtered_images = chapter_images[:min(3, len(chapter_images))]
        
        for img_data in filtered_images[:3]:  
            try:
                
                description = analyze_image_with_qwen(img_data['image'], content[:1000])
                
                
                if img_data['caption']:
                    full_caption = f"{img_data['caption']} - {description}"
                else:
                    full_caption = description
                
                processed_images.append({
                    'image': img_data['image'],
                    'caption': full_caption
                })
            except Exception as e:
                print(f"Error processing image for chapter {number}: {e}")
    
    return f"**CHAPTER {number}**\n\n{final_summary}", processed_images


def generate_sample_questions(chapter_text, chapter_images=None):
    
    image_context = ""
    if chapter_images and len(chapter_images) > 0:
        image_descriptions = [f"Image {i+1}: {img['caption']}" for i, img in enumerate(chapter_images)]
        image_context = "The chapter contains the following images:\n" + "\n".join(image_descriptions)
    
    prompt = f"""
    Generate exactly 15 educational questions based on the content with the following distribution:
    
    - 3 Multiple Choice Questions (MCQs) with options (A, B, C, D) and correct answers:
      * 1 MCQ that references one of the images
      * 2 MCQs based on text content
    
    - 5 Short Answer questions (2-3 marks):
      * 1 Short Answer question that references one of the images
      * 4 Short Answer questions based on text content
    
    - 5 Long Answer questions (5 marks):
      * 1 Long Answer question that references one of the images
      * 4 Long Answer questions based on text content
    
    - 2 Case-Based questions:
      * 1 Case-Based question that references one of the images
      * 1 Case-Based question based on text content
    
    {image_context}
    
    Format:
    MCQ: Question? (A) Option1 (B) Option2 (C) Option3 (D) Option4 | Answer: X
    MCQ: [IMAGE QUESTION] Question? (A) Option1 (B) Option2 (C) Option3 (D) Option4 | Answer: X
    Short Answer (2 marks): Question?
    Short Answer (2 marks): [IMAGE QUESTION] Question?
    Long Answer (5 marks): Question?
    Long Answer (5 marks): [IMAGE QUESTION] Question?
    Case-Based: Case description followed by questions
    Case-Based: [IMAGE QUESTION] Case description followed by questions
    
    For chemical equations, write them in plain text format (like "2H2 + O2 → 2H2O") not LaTeX format.
    """
    
    try:
        completion = client.chat.completions.create(
            model="deepseek/deepseek-r1:free",
            messages=[{"role": "user", "content": f"{prompt}\n\nContent: {chapter_text}"}],
            temperature=0.7
        )
        
        # Convert any LaTeX equations to normal format
        questions = completion.choices[0].message.content.strip().split("\n")
        normalized_questions = []
        
        for q in questions:
            # Convert LaTeX equations to normal text
            q = re.sub(r'\\\((.*?)\\\)', lambda m: m.group(1).replace('\\text{', '').replace('}', '').replace('\\', '').replace('_', ''), q)
            q = re.sub(r'\\left\((.*?)\\right\)', r'(\1)', q)
            q = re.sub(r'\\rightarrow', r'→', q)
            normalized_questions.append(q)
        
        return normalized_questions
    except Exception as e:
        return [f"Question generation failed: {e}"]   

def extract_questions(text):
    """Extract questions only from the EXERCISES section"""
    exercises_pattern = r"E\s*X\s*E\s*R\s*C\s*I\s*S\s*E\s*S(.*?)(?=\d+\s+CHAPTER|\Z)"
    matches = re.findall(exercises_pattern, text, re.DOTALL)
    
    questions = []
    for match in matches:
        exercises_section = match.strip()
        question_pattern = r"(\d+\.\s)(.*?)(?=\d+\.\s|\Z)"
        question_matches = re.findall(question_pattern, exercises_section)
        questions.extend([q[1].strip() for q in question_matches])
    
    return questions

def deepseek_answer(question, context, chapter_images=None):
    
    context_with_images = context
    if chapter_images and len(chapter_images) > 0:
        image_mentions = []
        for i, img_data in enumerate(chapter_images):
            if img_data.get('caption'):
                # Check if the question might be asking about figures
                if re.search(r'(figure|diagram|image|graph|chart|picture|plot|illustration)', 
                            question, re.IGNORECASE):
                    image_mentions.append(f"Figure {i+1}: {img_data['caption']}")
        
        if image_mentions:
            context_with_images += "\n\nRelevant Figures in the chapter:\n" + "\n".join(image_mentions)
    
    max_retries = 5
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model="deepseek/deepseek-r1:free",
                messages=[{
                    "role": "user", 
                    "content": f"""Answer this question in 2-3 sentences maximum. Just provide the direct answer without ANY reasoning process.
DO NOT write phrases like \"Based on the context\", \"the answer is\", etc.
DO NOT include any thinking process.

Question: {question}

Context from textbook: {context_with_images}"""
                }],
                temperature=0.3
            )
            answer = completion.choices[0].message.content.strip() if completion.choices else None
            
            if answer:
                answer = re.sub(r'^(Okay|Alright|Let me|Based on|Looking at|If we|To solve|In this|So|Now,|Well,|The question|From the|First,).*?[.!?]\s*', '', answer).strip()
                answer = re.sub(r'(The answer is|This means that|Thus|Therefore|Hence|So,|To summarize|In conclusion)[,:](.*?)$', r'\2', answer).strip()
                answer = re.sub(r'I (need to|will|should|can).*?[.!?]\s*', '', answer).strip()
                answer = re.sub(r'Let\'s.*?[.!?]\s*', '', answer).strip()
                answer = re.sub(r'Wait.*?[.!?]\s*', '', answer).strip()
                answer = re.sub(r'Hmm.*?[.!?]\s*', '', answer).strip()
                
                return answer
            
            time.sleep(1)
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
    
    return "This question requires further review."

def add_image_to_document(doc, image, caption=None, width=4.5):
    """Add an image to the Word document with optional caption"""
    try:
        
        img_byte_arr = BytesIO()
        image.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)
        
       
        doc.add_picture(img_byte_arr, width=Inches(width))
        
        
        if caption:
            caption_para = doc.add_paragraph(caption)
            caption_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
            caption_para.style = 'Caption'
        
        
        doc.add_paragraph()
        
        return True
    except Exception as e:
        print(f"Error adding image to document: {e}")
        return False

def format_text_with_bold(text):
    """Format text with bold for content between ** **"""
    
    bold_pattern = r'\*\*(.*?)\*\*'
    matches = re.findall(bold_pattern, text)
    
    
    for match in matches:
        text = text.replace(f"**{match}**", match)
    
    return text

def save_summary(summary_dict, images_dict, output_path="Summary.docx"):
    doc = Document()
    doc.add_heading("Chapter Summaries", level=1)

    for chapter, summary_text in summary_dict.items():
        
        if summary_text.startswith("**CHAPTER"):
            
            chapter_title = summary_text.split("\n")[0].replace("**", "")
            doc.add_heading(chapter_title, level=2)
            
            
            summary_content = "\n".join(summary_text.split("\n")[2:])
            
            
            for paragraph in summary_content.split("\n\n"):
                if paragraph.strip():
                    # Format the text - make content between ** ** bold
                    formatted_para = format_text_with_bold(paragraph.strip())
                    para = doc.add_paragraph()
                    
                    
                    bold_pattern = r'\*\*(.*?)\*\*'
                    parts = re.split(bold_pattern, formatted_para)
                    
                    for i, part in enumerate(parts):
                        
                        if i % 2 == 0:
                            para.add_run(part)
                        else:
                            para.add_run(part).bold = True
            
            
            chapter_images = images_dict.get(chapter, [])
            for img_data in chapter_images:
                add_image_to_document(doc, img_data['image'], img_data['caption'])
        else:
            
            doc.add_paragraph(summary_text)

    doc.save(output_path)
    return output_path


def save_questions(questions_dict, images_dict, output_path="Questions.docx"):
    doc = Document()
    doc.add_heading("Generated Questions", level=1)
    
    for chapter, questions in questions_dict.items():
        doc.add_heading(f"Chapter {chapter}", level=2)
        
        
        image_references = set()
        for question in questions:
            
            matches = re.findall(r'\bImage\s+(\d+)\b', question, re.IGNORECASE)
            matches.extend(re.findall(r'\[IMAGE\s+(\d+)\]', question, re.IGNORECASE))
            matches.extend(re.findall(r'\[IMAGE QUESTION\]', question, re.IGNORECASE))
            
            
            for match in matches:
                if match.isdigit():
                    image_references.add(int(match))
                else:
                    # For general [IMAGE QUESTION] tags, include all images
                    image_references.update(range(1, len(images_dict.get(chapter, [])) + 1))
        
        
        chapter_images = images_dict.get(chapter, [])
        if chapter_images:
            doc.add_paragraph("Chapter Visual References:")
            
            
            for i, img_data in enumerate(chapter_images):
                
                if (i+1) in image_references or not image_references:
                    doc.add_paragraph(f"Image {i+1}:")
                    add_image_to_document(doc, img_data['image'], img_data['caption'])
        
       
        doc.add_heading("Questions", level=3)
        for question in questions:
            if not question.strip():
                continue
                
            
            question = re.sub(r'\\\((.*?)\\\)', lambda m: m.group(1).replace('\\text{', '').replace('}', '').replace('\\', '').replace('_', ''), question)
            question = re.sub(r'\\left\((.*?)\\right\)', r'(\1)', question)
            question = re.sub(r'\\rightarrow', r'→', question)
            
            
            paragraph = doc.add_paragraph()
            
            
            if question.strip().startswith('-'):
                # Replace with bullet
                question = "•" + question[1:]
            
            
            if '**' in question:
                
                parts = re.split(r'(\*\*.*?\*\*)', question)
                for part in parts:
                    if part.startswith('**') and part.endswith('**'):
                        
                        bold_text = part[2:-2]
                        paragraph.add_run(bold_text).bold = True
                    else:
                        
                        paragraph.add_run(part)
            else:
                
                paragraph.add_run(question)
            
            
            doc.add_paragraph()
    
    doc.save(output_path)
    return output_path         
        

def save_qa(chapter_dict, qa_dict, images_dict, output_path="NCERT_Questions_Answers.docx"):
    doc = Document()
    doc.add_heading("NCERT Questions and Answers", level=1)

    for chapter, content in chapter_dict.items():
        doc.add_page_break()
        doc.add_heading(f"Chapter {chapter}", level=2)
        
        
        chapter_images = images_dict.get(chapter, [])

        for i, question in enumerate(qa_dict.get(chapter, [])):
            
            is_image_question = False
            relevant_image = None
            
            if chapter_images and re.search(r'(figure|diagram|image|graph|chart|picture|plot|illustration)', 
                                          question, re.IGNORECASE):
                is_image_question = True
                
                for img_data in chapter_images:
                    if img_data.get('caption') and any(word in img_data['caption'].lower() 
                                                     for word in question.lower().split()):
                        relevant_image = img_data
                        break
                
                if not relevant_image and chapter_images:
                    relevant_image = chapter_images[0]  
            
           
            question_para = doc.add_paragraph()
            question_para.add_run("Q: ").bold = True
            
            
            if '**' in question:
                parts = re.split(r'(\*\*.*?\*\*)', question)
                for part in parts:
                    if part.startswith('**') and part.endswith('**'):
                        # Bold text
                        bold_text = part[2:-2]
                        question_para.add_run(bold_text).bold = True
                    else:
                        
                        question_para.add_run(part)
            else:
                question_para.add_run(question)
            
            
            if is_image_question and relevant_image:
                add_image_to_document(doc, relevant_image['image'], width=3.5)
            
            
            answer = deepseek_answer(question, content, chapter_images)
            
            
            answer = re.sub(r'\\\((.*?)\\\)', lambda m: m.group(1).replace('\\text{', '').replace('}', '').replace('\\', '').replace('_', ''), answer)
            
            answer_para = doc.add_paragraph()
            answer_para.add_run("A: ").bold = True
            
            
            if '**' in answer:
                parts = re.split(r'(\*\*.*?\*\*)', answer)
                for part in parts:
                    if part.startswith('**') and part.endswith('**'):
                        # Bold text
                        bold_text = part[2:-2]
                        answer_para.add_run(bold_text).bold = True
                    else:
                        # Normal text
                        answer_para.add_run(part)
            else:
                answer_para.add_run(answer)
            
            if i < len(qa_dict.get(chapter, [])) - 1:
                doc.add_paragraph("")
    
    doc.save(output_path)
    return output_path

def match_figures_to_chapters(figures, chapters):
    """Match figures to chapters based on text content"""
    chapter_images = {chapter: [] for chapter in chapters.keys()}
    
    
    print("Analyzing figures and matching to chapters...")
    for img_data in figures:
        try:
            
            description = analyze_image_with_qwen(img_data['image'])
            
          
            if img_data['caption']:
                full_caption = f"{img_data['caption']} - {description}"
            else:
                full_caption = description
                
            img_data['caption'] = full_caption
            
            
            best_chapter = None
            highest_score = 0
            
            
            desc_words = set(re.findall(r'\b\w+\b', description.lower()))
            
            for chapter, content in chapters.items():
                
                chapter_sample = content[:5000].lower()
                
                
                match_score = sum(1 for word in desc_words if word in chapter_sample and len(word) > 3)
                
                if match_score > highest_score:
                    highest_score = match_score
                    best_chapter = chapter
            
            
            if best_chapter:
                chapter_images[best_chapter].append(img_data)
            else:
                
                first_chapter = list(chapters.keys())[0] if chapters else None
                if first_chapter:
                    chapter_images[first_chapter].append(img_data)
        except Exception as e:
            print(f"Error analyzing figure {img_data.get('filename', 'unknown')}: {e}")
    
    return chapter_images

def process_files(pdf_files, figures_folder, output_dir="./output", generate_summary=True, 
                 generate_questions=True, generate_qa=True):
    """Process PDF files and figures folder"""
    
    os.makedirs(output_dir, exist_ok=True)
    
    
    output_files = []
    all_chapters = {}
    all_qa_chapters = {}
    all_questions = {}
    
   
    all_figures = load_figures_from_folder(figures_folder)
    
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        
        futures = []
        for pdf_file in pdf_files:
            future = executor.submit(extract_text_from_pdf, pdf_file)
            futures.append(future)
        
        
        for future in futures:
            text = future.result()
            
            
            if generate_summary or generate_questions:
                chapters = extract_chapters(text, for_qa=False)
                all_chapters.update(chapters)
            
            if generate_qa:
                qa_chapters = extract_chapters(text, for_qa=True)
                for chapter, content in qa_chapters.items():
                    questions = extract_questions(content)
                    all_questions[chapter] = questions
                    all_qa_chapters[chapter] = content
    
    
    chapter_figures = match_figures_to_chapters(all_figures, all_chapters)
    
    
    if generate_summary:
        summary_results = {}
        processed_images = {}
        
        with ThreadPoolExecutor(max_workers=4) as executor:
            
            future_to_chapter = {
                executor.submit(summarize_chapter, chapter, content, chapter_figures.get(chapter, [])): chapter
                for chapter, content in all_chapters.items()
            }
            
            
            for future in future_to_chapter:
                chapter = future_to_chapter[future]
                summary_text, chapter_processed_images = future.result()
                summary_results[chapter] = summary_text
                processed_images[chapter] = chapter_processed_images

        
        summary_path = os.path.join(output_dir, "Summary.docx")
        save_summary(summary_results, processed_images, summary_path)
        output_files.append(summary_path)
    
  
    if generate_questions:
        questions_dict = {}
        
        with ThreadPoolExecutor(max_workers=4) as executor:
           
            future_to_chapter = {
                executor.submit(generate_sample_questions, 
                               content, 
                               chapter_figures.get(chapter, [])): chapter
                for chapter, content in all_chapters.items()
            }
            
            
            for future in future_to_chapter:
                chapter = future_to_chapter[future]
                questions_dict[chapter] = future.result()

        
        questions_path = os.path.join(output_dir, "Questions.docx")
        save_questions(questions_dict, chapter_figures, questions_path)
        output_files.append(questions_path)
    
    
    if generate_qa:
        qa_path = os.path.join(output_dir, "NCERT_Questions_Answers.docx")
        save_qa(all_qa_chapters, all_questions, chapter_figures, qa_path)
        output_files.append(qa_path)
    
    return output_files

def main(pdf_files_paths, figures_folder, output_dir=None):
    """Main function to process PDF files and figures"""
    if output_dir is None:
        
        if os.path.exists('/kaggle/working'):
            output_dir = '/kaggle/working'
        else:
            output_dir = './output'

    print(f"Processing {len(pdf_files_paths)} PDF files with figures from {figures_folder}...")
    output_files = process_files(
        pdf_files_paths,
        figures_folder,
        output_dir=output_dir,
        generate_summary=False,
        generate_questions=True, 
        generate_qa=False
    )
    
    print(f"Processing complete. Output files saved to: {output_dir}")
    for file in output_files:
        print(f" - {os.path.basename(file)}")
    
    return output_files

if __name__ == "__main__":
    
    pdf_files = ["/kaggle/input/books-pdf/Books/jesc110.pdf"]
    figures_folder = "/kaggle/input/d/smritikc/figure/Figure10"  
    
    
    output_directory = '/kaggle/working' if os.path.exists('/kaggle/working') else './output'
    main(pdf_files, figures_folder, output_directory)